Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.decomposition import PCA

In [2]:
xls = pd.ExcelFile('Data.xlsx')
df = xls.parse(0)
factors_df = xls.parse(1)

In [3]:
df['Time'] = pd.to_datetime(df['Time'])
factors_df['Time'] = pd.to_datetime(factors_df['Time'])

In [4]:
df.head()

,Time,GDPC1,PCECTPI,TB3MS,GS10
0,1960-06-30,3430.057,15.331,3.5400,4.5033
1,1960-09-30,3439.832,15.415,4.2300,4.5833
2,1960-12-30,3517.181,15.435,3.8733,4.4867
3,1961-03-30,3498.246,15.515,2.9933,4.2600
4,1961-06-30,3515.385,15.574,2.3600,3.8333


Create $\log(GDPC1)$ and $\Delta \log(GDPC1)$

In [5]:
df['log_GDP'] = np.log(df['GDPC1'])
df['delta_log_GDP'] = (df['log_GDP'] - df['log_GDP'].shift(1))

df['delta_log_GDP'].fillna(0, inplace=True)

Create $\pi$ and $\log(PCECTPI)$

In [6]:
df['infl'] = (np.log(df['PCECTPI']) - (np.log(df['PCECTPI'])).shift(1))
df['infl'].fillna(0, inplace=True)

df['log_PCECTPI'] = np.log(df['PCECTPI'])

In [7]:
df.head()

,Time,GDPC1,PCECTPI,TB3MS,GS10,log_GDP,delta_log_GDP,infl,log_PCECTPI
0,1960-06-30,3430.057,15.331,3.5400,4.5033,8.140332,0.000000,0.000000,2.729877
1,1960-09-30,3439.832,15.415,4.2300,4.5833,8.143178,0.002846,0.005464,2.735341
2,1960-12-30,3517.181,15.435,3.8733,4.4867,8.165415,0.022237,0.001297,2.736638
3,1961-03-30,3498.246,15.515,2.9933,4.2600,8.160017,-0.005398,0.005170,2.741807
4,1961-06-30,3515.385,15.574,2.3600,3.8333,8.164904,0.004887,0.003796,2.745603


Create $TSpread$

In [8]:
df['tspread'] = df['GS10'] - df['TB3MS']

### **Calculate Principal Component**

Create `stand_factors` as a data frame containing 245 columns, namely all the columns of factors_df except the _Time_, but standardized ($\mu =0$, $\sigma=1$).

`factors_df` doesn't change.

In [9]:
stand_factors = factors_df.iloc[:, 1:].apply(stats.zscore, axis=0)

In [10]:
stand_factors.head()

,PCECC96,PCDGx,PCESVx,PCNDx,GPDIC1,FPIx,Y033RC1Q027SBEAx,PNFIx,PRFIx,A014RE1Q156NBEA,...,TLBSNNBx,TLBSNNBBDIx,TABSNNBx,TNWBSNNBx,TNWBSNNBBDIx,CNCFx,S&P 500,S&P: indust,S&P div yield,S&P PE ratio
0,0.345487,0.198582,1.001568,-0.275298,-1.826943,0.057890,0.639820,0.732729,-0.498651,-0.640085,...,0.977503,-0.506133,-0.828227,-0.771472,-0.749628,0.172105,0.074484,0.099485,-0.115484,-0.174575
1,-0.950126,-2.016541,0.743634,-0.003276,0.545897,-0.965475,-0.488568,-0.684404,-0.716405,0.536824,...,-2.142299,-0.851033,-1.020656,-0.485978,-1.933865,0.875186,-0.549149,-0.561475,0.440121,-0.612021
2,0.223639,0.605363,0.192954,-0.425468,2.077526,1.086674,0.710134,1.098489,0.589866,2.722511,...,1.900739,-0.445382,0.026075,-0.209566,1.850410,-1.008960,-0.706124,-0.779212,1.132685,-0.570060
3,0.645074,0.307674,0.557259,0.550779,-2.573436,-1.273241,0.132059,0.151842,-1.972480,0.200564,...,0.429553,-0.428397,-0.528344,-0.456013,-0.430267,0.102296,-0.335165,-0.399775,0.451777,-0.242132
4,-1.667558,-0.685054,-1.632966,-1.404069,-0.284477,-1.426275,-1.882351,-1.400137,-0.752689,0.536824,...,0.462580,-0.513322,-0.082459,-0.096851,-0.948319,0.470454,-0.377939,-0.490168,0.160376,0.013255


In [11]:
pca = PCA(n_components=1)
first_PC = pca.fit_transform(stand_factors)

Da fare per ogni rolling window, 134 volte dio cane

## Models

### Random Walk

This is just a proposal of a function that works for Random Walk and AR(4), i think it can be implemented for other models.
Anyway it may be used as a blueprint to handle the rolling windows

In [ ]:
def forecast(q, p, d, estimated_series): 
    
    # q, p, d are the parameters of the ARIMA model
    # estimated_series = a string with the name of the series we want to forecast
    # wind_initial_origin/end = index of the first and last observation for the 
    #                           the first rolling window (0:100)
    # wind_current_origin/end = index to be updated at each iteration
    # forecasts = empty series to store the forecasts
    

    forecasts = pd.Series(dtype = 'float64')

    wind_initial_origin = 0
    wind_intial_end = 100 

    wind_current_origin = wind_initial_origin
    wind_current_end = wind_initial_end 

    for t in range(len(dtest.index)):

        current_window = df.iloc[wind_current_origin:wind_current_end] # from the original df we slice in order
                                                                       # to select only the current window

        model = ARIMA(current_window[estimated_series], order = (q, p, d))
        fitted_model = model.fit()

        current_forecast = fitted_model.forecast(steps=1)

        forecasts= pd.concat([forecasts, current_forecast])

        wind_current_origin +=1
        wind_current_end += 1


    return forecasts